In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import colorgram
import bandcamp_webtools as bw

In [3]:
bw.cowdog('https://chillhop.bandcamp.com/track/after-dark', loops=3, note_mon=True)

Current Artist Count: 1
Current Artist Count: 7
Current Artist Count: 10
Current Artist Count: 15
Current Artist Count: 19


KeyboardInterrupt: 

In [ ]:
done = False
while not done:
    try:
        done = album_cover_scrape()
    except requests.exceptions.ConnectionError:
        sleep(5)